
### **Cell 1: Imports and Initial Setup**

This cell imports the necessary libraries and installs `kaleido` if needed, which is required for exporting plots to static image files.



In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
import os

# Kaleido is needed for exporting plotly figures to static images.
%pip install -U kaleido

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.




### **Cell 2: Configuration**

This cell centralizes all configurations, including paths, algorithm groups, and metric names, making the script easier to manage.



In [ ]:
# Base path for algorithm results
BASE_RESULTS_PATH = r'..\..\..\results\base_experiments\algorithms'

# Dictionary mapping conversion methods to their respective algorithms and result file paths
ALGORITHMS_BY_METHOD = {
    'ca': {
        'DeepSVDD_ca': r'DeepSVDD\ca\CA.csv',
        'FastABOD_ca': r'FastABOD\ca\CA.csv',
        'iForest_ca': r'iForest\ca\CA.csv',
        'KNN_ca': r'KNN\ca\CA.csv',
        'LOF_ca': r'LOF\ca\CA.csv',
        'McCatch_ca': r'McCatch\ca\CA.csv',
    },
    'idf': {
        'DeepSVDD_idf': r'DeepSVDD\idf\IDF.csv',
        'FastABOD_idf': r'FastABOD\idf\IDF.csv',
        'iForest_idf': r'iForest\idf\IDF.csv',
        'KNN_idf': r'KNN\idf\IDF.csv',
        'LOF_idf': r'LOF\idf\IDF.csv',
        'McCatch_idf': r'McCatch\idf\IDF.csv',
    },
    'onehot': {
        'DeepSVDD_one': r'DeepSVDD\one_hot\ONE_HOT.csv',
        'FastABOD_one': r'FastABOD\one_hot\ONE_HOT.csv',
        'iForest_one': r'iForest\one_hot\ONE_HOT.csv',
        'KNN_one': r'KNN\one_hot\ONE_HOT.csv',
        'LOF_one': r'LOF\one_hot\ONE_HOT.csv',
        'McCatch_one': r'McCatch\one_hot\ONE_HOT.csv',
    },
    'pivot': {
        'DeepSVDD_pivot': r'DeepSVDD\pivot\PIVOT.csv',
        'FastABOD_pivot': r'FastABOD\pivot\PIVOT.csv',
        'iForest_pivot': r'iForest\pivot\PIVOT.csv',
        'KNN_pivot': r'KNN\pivot\PIVOT.csv',
        'LOF_pivot': r'LOF\pivot\PIVOT.csv',
        'McCatch_pivot': r'McCatch\pivot\PIVOT.csv',
    },
    'nocat': {
        'DeepSVDD_nocat': r'DeepSVDD\nocat\NOCAT.csv',
        'FastABOD_nocat': r'FastABOD\nocat\NOCAT.csv',
        'iForest_nocat': r'iForest\nocat\NOCAT.csv',
        'KNN_nocat': r'KNN\nocat\NOCAT.csv',
        'LOF_nocat': r'LOF\nocat\NOCAT.csv',
        'McCatch_nocat': r'McCatch\nocat\NOCAT.csv',
    }
}

# List of metrics to be analyzed
METRICS = ['auc', 'adj_r_precision', 'adj_average_precision', 'adj_max_f1']

# Mapping from metric names to display names for plots
METRIC_DISPLAY_NAMES = {
    'auc': 'AUC',
    'adj_r_precision': 'P@n',
    'adj_average_precision': 'AP',
    'adj_max_f1': 'Max-F1'
}



### **Cell 3: Load Dataset Metadata**

This cell loads the summary file containing metadata for all datasets to create a master list of datasets to be included in the analysis.



In [ ]:
# Load the summary file containing metadata for all datasets
df_summary = pd.read_csv(r'..\..\resume_datasets.csv', sep=';')

# Get a sorted, unique list of all dataset file names
ALL_DATASET_NAMES = sorted(df_summary['file'].str.replace('.csv', '').unique().tolist())
print(f"Loaded metadata for {len(ALL_DATASET_NAMES)} datasets.")



### **Cell 4: Data Processing Helper Functions**

This cell contains reusable functions for cleaning and preparing the performance data before plotting.



In [ ]:
def average_dataset_versions(df, metrics_to_average):
    """
    Averages results for datasets that have multiple versions (e.g., _v01, _v02).
    It standardizes the dataset name and then groups by dataset and parameter to average the scores.
    """
    if 'dataset' not in df.columns:
        return df
        
    # Standardize dataset names by removing version suffixes (e.g., _v01)
    df['dataset'] = df['dataset'].str.replace(r'_v[0-9]{1,2}', '', regex=True)
    
    # Define columns to group by
    grouping_cols = ['dataset', 'parameter', 'algorithm']
    
    # Calculate the mean for the specified metric columns
    df_averaged = df.groupby(grouping_cols)[metrics_to_average].mean().reset_index()
    
    return df_averaged

def fill_missing_scores(series, metric_name):
    """Fills NaN values in a series with a default score based on the metric."""
    default_scores = {
        'auc': 0.5,
        'adj_r_precision': 0.0,
        'adj_average_precision': 0.0,
        'adj_max_f1': 0.0
    }
    return series.fillna(default_scores.get(metric_name, 0.0))

def filter_by_dataset_list(df, dataset_list):
    """Filters a DataFrame to keep only the datasets present in the provided list."""
    # Standardize dataset names before filtering
    df['dataset'] = df['dataset'].str.replace(r'(_v[0-9]{1,2})?\.csv$', '', regex=True)
    return df[df['dataset'].isin(dataset_list)].copy()



### **Cell 5: Plotting Function**

This cell defines the main function responsible for creating, styling, and saving the boxplots.



In [ ]:
def generate_boxplot(dataframe, value_col, category_col, metric_name, title='Boxplot', save=True):
    """
    Generates and displays a boxplot using Plotly, with options to save the figure.
    It also removes outliers based on the IQR method before plotting.
    """
    # Define a consistent color map for the conversion methods
    color_map = {
         "ca": "#636EFA",
         "idf": "#00CC96",
         "onehot": "#FFA15A",
         "pivot": "#AB63FA",
         "nocat": "#EF553B"
    }
    
    # --- Remove outliers using the IQR method ---
    # This step is done to improve the readability of the boxplot visualization.
    Q1 = dataframe[value_col].quantile(0.25)
    Q3 = dataframe[value_col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    df_filtered = dataframe[(dataframe[value_col] >= lower_bound) & (dataframe[value_col] <= upper_bound)]
    
    # Create the boxplot figure
    fig = px.box(
        df_filtered, 
        x=category_col, 
        y=value_col, 
        title=title, 
        color="method", 
        color_discrete_map=color_map
    )
    
    # Update layout for a cleaner, more professional look
    fig.update_layout(
        boxmode='group',
        title=None,
        xaxis_title='Conversion Methods',
        yaxis_title=metric_name,
        showlegend=False,
        font=dict(size=14),
        xaxis=dict(title_font=dict(size=16)),
        yaxis=dict(title_font=dict(size=16))
    )
    fig.update_traces(width=0.6) # Adjust box width
    
    # --- Add Median Annotations ---
    # Calculate medians for each category to display on the plot
    medians = df_filtered.groupby('method')[value_col].median().round(4)
    for category, median_val in medians.items():
        fig.add_annotation(
            x=category,
            y=median_val,
            text=f"{median_val}",
            showarrow=False,
            yshift=10, # Shift text slightly above the median line
            font=dict(color='black', size=12)
        )
        
    fig.show()
    
    # Save the figure if requested
    if save:
        output_dir = r'..\..\..\results\base_experiments\plot\BOXPLOT'
        os.makedirs(output_dir, exist_ok=True)
        output_path = os.path.join(output_dir, f"boxplot_{metric_name.lower()}.png")
        fig.write_image(output_path)
        print(f"Saved plot to {output_path}")



### **Cell 6: Main Processing and Plot Generation**

This is the main execution block. It iterates through each metric, loads the corresponding data, processes it using the helper functions, and generates a boxplot comparing the different conversion methods.



In [ ]:
# This cell processes the results for the 'average' performance case.
# It calculates the average performance of each conversion method across all its algorithms for each dataset.

for metric in METRICS:
    print(f"--- Processing metric: {metric} ---")
    
    all_methods_data = []
    
    # Iterate over each conversion method (ca, idf, etc.)
    for method, algorithms in ALGORITHMS_BY_METHOD.items():
        
        all_algos_in_method = []
        
        # Load data for each algorithm within the current method
        for algo_name, file_path in algorithms.items():
            full_path = os.path.join(BASE_RESULTS_PATH, file_path)
            if os.path.exists(full_path):
                try:
                    df_algo = pd.read_csv(full_path, sep=';')
                    # Standardize dataset names and filter to keep only relevant ones
                    df_algo = filter_by_dataset_list(df_algo, ALL_DATASET_NAMES)
                    # Average results for datasets with multiple versions
                    df_algo = average_dataset_versions(df_algo, [metric])
                    all_algos_in_method.append(df_algo)
                except Exception as e:
                    print(f"Could not process file {full_path}: {e}")
            else:
                print(f"File not found, skipping: {full_path}")

        # If no data was loaded for this method, skip to the next one
        if not all_algos_in_method:
            continue
            
        # Combine data from all algorithms in the current method
        df_method = pd.concat(all_algos_in_method, ignore_index=True)
        
        # Calculate the average score for the method on each dataset
        # This averages the performance of all algorithms (DeepSVDD, iForest, etc.) for that method
        df_method_avg = df_method.groupby('dataset')[metric].mean().reset_index()
        
        # Add a column to identify the method
        df_method_avg['method'] = method
        
        all_methods_data.append(df_method_avg)

    # Combine the averaged data from all methods into a final DataFrame
    if not all_methods_data:
        print(f"No data found for metric {metric}. Skipping plot generation.")
        continue
        
    df_final_for_plot = pd.concat(all_methods_data, ignore_index=True)
    
    # Rename the metric column to a generic 'value' for the plotting function
    df_final_for_plot.rename(columns={metric: 'value'}, inplace=True)
    
    # Fill any missing values with the appropriate default for the metric
    df_final_for_plot['value'] = fill_missing_scores(df_final_for_plot['value'], metric)
    
    # Generate the boxplot for the current metric
    display_name = METRIC_DISPLAY_NAMES.get(metric, metric)
    generate_boxplot(
        df_final_for_plot, 
        value_col='value', 
        category_col='method', 
        metric_name=display_name, 
        title=f'Conversion Method Performance ({display_name})'
    )

print("\n--- All plots generated. ---")